In [17]:
import numpy as np
from numpy.linalg import norm
from scipy.sparse.linalg import cg as sp_cg

def cg_basic(A, b, x0=None, rtol=1e-8, atol=0.0, maxiter=None, callback=None):
    A = np.asarray(A, float)
    n = A.shape[0]
    x = np.zeros(n) if x0 is None else np.asarray(x0, float).copy()

    b_norm = norm(b)
    tol = max(rtol * b_norm, atol)
    r = b - A @ x
    p = r.copy()
    rr_old = float(r @ r)
    res_hist = [np.sqrt(rr_old)]
    if maxiter is None:
        maxiter = 5 * n

    for k in range(1, maxiter + 1):
        Ap = A @ p
        alpha = rr_old / float(p @ Ap)
        x += alpha * p
        r -= alpha * Ap
        rr_new = float(r @ r)
        res = np.sqrt(rr_new)
        res_hist.append(res)
        if callback is not None:
            callback(x)
        if res <= tol:
            return x, 0, res_hist
        beta = rr_new / rr_old
        p = r + beta * p
        rr_old = rr_new

    return x, k, res_hist

In [18]:
rng = np.random.default_rng(0)
n = 200
Q = rng.standard_normal((n, n))
A = Q.T @ Q + 1e-3 * np.eye(n)  # SPD
x_true = rng.standard_normal(n)
b = A @ x_true

In [19]:
x_cg, info_cg, hist = cg_basic(A, b, rtol=1e-8, atol=0.0, maxiter=5000)

res_hist_sp = []
def cb(xk):
    res_hist_sp.append(norm(b - A @ xk))

x_sp, info_sp = sp_cg(A, b, rtol=1e-8, atol=0.0, maxiter=5000, callback=cb)

print("basic CG: info=", info_cg, "iters≈", len(hist)-1,
      "final ||r||=", hist[-1], "rel err=", norm(x_cg - x_true)/norm(x_true))
print("SciPy  CG: info=", info_sp, "iters≈", len(res_hist_sp),
      "final ||r||=", (res_hist_sp[-1] if res_hist_sp else norm(b - A @ x_sp)),
      "rel err=", norm(x_sp - x_true)/norm(x_true))
print("Solutions close? ", np.allclose(x_cg, x_sp, rtol=1e-6, atol=1e-10))

basic CG: info= 0 iters≈ 326 final ||r||= 1.608903536384652e-05 rel err= 4.478405910942444e-09
SciPy  CG: info= 0 iters≈ 326 final ||r||= 1.6089035545558882e-05 rel err= 4.478405910942444e-09
Solutions close?  True
